In [12]:
import numpy as np
from methods.regression_methods import NGGP
from run_regression import *
from training.configs import Config as config
from models import backbone
import torch
from argparse import Namespace
from data.neural_loader import *


In [42]:
save_path_th = '/Users/jessegill/Desktop/nggp/nggp_lib/save/nggp_rbf_neural --kernel_type rbf --stop_epoch 50000 --all_lr 1e-3 --use_conditional True --context_type backbone /checkpoints/neural/MLP2_NGGP_model.th'
data = np.load('/Users/jessegill/Desktop/nggp/nggp_lib/save/nggp_rbf_neural --kernel_type rbf --stop_epoch 50000 --all_lr 1e-3 --use_conditional True --context_type backbone /checkpoints/neural/MLP2_NGGP_model.th', allow_pickle = True)
params = Namespace(seed=1, model='MLP2', method='NGGP', dataset='neural', update_batch_size=5, meta_batch_size=5, output_dim=40, multidimensional_amp=False, multidimensional_phase=False, noise='gaussian', kernel_type='rbf', save_dir='./save/nggp_rbf_neural --kernel_type rbf --stop_epoch 50000 --all_lr 1e-3 --use_conditional True --context_type backbone ', num_tasks=1, multi_type=3, method_lr=0.001, feature_extractor_lr=0.001, cnf_lr=0.001, all_lr=None, neptune=False, use_conditional=True, context_type='backbone', layer_type='concatsquash', dims='32-32', num_blocks=2, time_length=0.5, train_T=False, add_noise=False, divergence_fn='brute_force', nonlinearity='tanh', solver='dopri5', atol=1e-05, rtol=1e-05, step_size=None, test_solver=None, test_atol=None, test_rtol=None, residual=False, rademacher=False, spectral_norm=False, batch_norm=False, bn_lag=0, l1int=None, l2int=None, dl2int=None, JFrobint=None, JdiagFrobint=None, JoffdiagFrobint=None, start_epoch=0, stop_epoch=100, test=False, n_support=5, n_test_epochs=10, out_of_range=False, device='cuda')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jessegill/Desktop/nggp/nggp_lib/save/nggp_rbf_neural --kernel_type rbf --stop_epoch 50000 --all_lr 1e-3 --use_conditional True --context_type backbone /checkpoints/neural/MLP2_NGGP_model.th'

In [ ]:
'/Users/jessegill/Desktop/nggp/nggp_lib/save/nggp_rbf_neural --kernel_type rbf --stop_epoch 50000 --all_lr 1e-3 --use_conditional True --context_type backbone 


/checkpoints/neural/MLP2_NGGP_model.th'

In [7]:
run = False
setup_seed(params)
config = Config(params)
checkpoint_dir, save_path = setup_checkpoint_dir(params)

results_logger = ResultsLogger(params)


device = 'cpu'
logging.info('Device: {}'.format(device))

bb = setup_backbone(device, params)
model = setup_model(bb, config, device, params)
optimizer = setup_optimizer(model, params)

if params.test and run:
    test(model, params, save_path, results_logger)
elif run:
    train(model, optimizer, params, save_path, results_logger)
model.load_checkpoint(save_path_th, device)

Device type: <class 'str'>
Device type: <class 'str'>


In [33]:
neural_loader = NeuralDatasetLoader()
batch = neural_loader.get_batch()

In [40]:
run = True
params = Namespace(seed=1, model='MLP2', method='NGGP', dataset='neural', update_batch_size=5, meta_batch_size=5, output_dim=40, multidimensional_amp=False, multidimensional_phase=False, noise='gaussian', kernel_type='rbf', save_dir='./save/nggp_rbf_neural --kernel_type rbf --stop_epoch 50000 --all_lr 1e-3 --use_conditional True --context_type backbone ', num_tasks=1, multi_type=3, method_lr=0.001, feature_extractor_lr=0.001, cnf_lr=0.001, all_lr=None, neptune=False, use_conditional=True, context_type='backbone', layer_type='concatsquash', dims='32-32', num_blocks=2, time_length=0.5, train_T=False, add_noise=False, divergence_fn='brute_force', nonlinearity='tanh', solver='dopri5', atol=1e-05, rtol=1e-05, step_size=None, test_solver=None, test_atol=None, test_rtol=None, residual=False, rademacher=False, spectral_norm=False, batch_norm=False, bn_lag=0, l1int=None, l2int=None, dl2int=None, JFrobint=None, JdiagFrobint=None, JoffdiagFrobint=None, start_epoch=0, stop_epoch=100,  n_support=5, n_test_epochs=10, out_of_range=False, device='cuda', test = True)
setup_seed(params)
config = Config(params)
checkpoint_dir, save_path = setup_checkpoint_dir(params)

results_logger = ResultsLogger(params)


device = 'cpu'
logging.info('Device: {}'.format(device))

bb = setup_backbone(device, params)
model = setup_model(bb, config, device, params)
optimizer = setup_optimizer(model, params)

if params.test and run:
    test(model, params, save_path, results_logger)
elif run:
    train(model, optimizer, params, save_path, results_logger)




Device type: <class 'str'>
Device type: <class 'str'>


FileNotFoundError: [Errno 2] No such file or directory: './save/nggp_rbf_neural --kernel_type rbf --stop_epoch 50000 --all_lr 1e-3 --use_conditional True --context_type backbone /checkpoints/neural/MLP2_NGGP_model.th'